In [1]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.applications import VGG16
from art.attacks.evasion import HopSkipJump
from art.estimators.classification import TensorFlowV2Classifier
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [2]:
# Step 1: Load the Oxford Flowers 102 dataset
(ds_train, ds_test), ds_info = tfds.load(
    'oxford_flowers102',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

# Step 2: Preprocess the data
def preprocess_image(image, label):
    # Resize images to 224x224 as VGG16 expects this input size
    image = tf.image.resize(image, (224, 224)) / 255.0  # Normalize to [0,1]
    return image, label

# Apply preprocessing to the dataset
ds_train = ds_train.map(preprocess_image).batch(64).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.map(preprocess_image).batch(64).prefetch(tf.data.AUTOTUNE)

In [3]:
# Step 3: Load the pretrained VGG16 model and freeze its layers
vgg16_base = VGG16(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
vgg16_base.trainable = False  # Freeze the VGG16 layers

# Step 4: Add custom layers on top of VGG16
model = Sequential([
    vgg16_base,  # Pretrained base model
    GlobalAveragePooling2D(),  # Pooling layer to reduce dimensionality
    Dense(256, activation='relu'),  # Fully connected layer
    Dense(102, activation='softmax')  # Output layer for 102 classes
])

# Step 5: Compile the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Print the model summary
print("\nModel Summary:")
model.summary()


Model Summary:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 102)            │        26,214 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,872,230 (56.73 MB)

 Trainable params: 157,542 (615.40 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [ ]:
# Step 6: Train the model on the Oxford Flowers 102 dataset
print("\nTraining the model...")
history = model.fit(ds_train, validation_data=ds_test, epochs=1)

# Visualize training progress
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy Over Epochs')
plt.show()


Training the model...
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.0155 - loss: 4.5897

In [ ]:
# Step 7: Wrap the model using ART's TensorFlowV2Classifier
classifier = TensorFlowV2Classifier(
    model=model,
    input_shape=(224, 224, 3),
    nb_classes=102,
    clip_values=(0.0, 1.0)  # Since images are normalized between [0, 1]
)

In [ ]:
# Step 8: Get one sample image from the test set
for x_sample, y_sample in ds_test.take(1):  # Take one batch
    x_sample = x_sample[:1]  # Take one image
    y_sample = y_sample[:1]  # Take its corresponding label

# Convert TensorFlow tensors to NumPy arrays
x_sample = x_sample.numpy()
y_sample = y_sample.numpy()

# Visualize the original image and its true label
print("\nOriginal Image and True Label:")
plt.imshow(x_sample.reshape(224, 224, 3))
plt.axis('off')
plt.show()
print(f"True label for the image: {y_sample[0]}")

In [ ]:
# Step 9: Choose a target class for misclassification
# Let's choose a target class different from the original class
target_class = np.array([5])  # For example, we want the model to misclassify this as class 5

# Convert the target class to one-hot encoding
target_class_one_hot = tf.keras.utils.to_categorical(target_class, num_classes=102)

# Print the chosen target class for misclassification
print(f"\nTarget class chosen for misclassification: {target_class[0]}")

In [ ]:
# Step 10: Perform the targeted HopSkipJump attack
print("\nRunning the targeted HopSkipJump attack...")
attack = HopSkipJump(classifier=classifier, targeted=True, max_iter=10)
x_adv = attack.generate(x=x_sample, y=target_class_one_hot)  # Provide the target class for targeted misclassification

# Visualize the adversarial image
print("\nAdversarial Image Generated:")
plt.imshow(x_adv.reshape(224, 224, 3))
plt.axis('off')
plt.show()

In [ ]:
# Step 11: Check the model's prediction before and after the attack
pred_before = np.argmax(classifier.predict(x_sample), axis=1)
pred_after = np.argmax(classifier.predict(x_adv), axis=1)

print(f"\nModel's prediction before attack: {pred_before[0]}")
print(f"Model's prediction after targeted attack: {pred_after[0]} (target: {target_class[0]})")

In [ ]:
# Step 12: Visualize the original vs. adversarial images side by side
print("\nOriginal vs. Adversarial Image Comparison:")
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(x_sample.reshape(224, 224, 3))
plt.axis("off")

plt.subplot(1, 2, 2)
plt.title("Adversarial Image")
plt.imshow(x_adv.reshape(224, 224, 3))
plt.axis("off")
plt.show()